# Example use of the Sonic module Linckii.jl

In [1]:
import Pkg                      # package manager
Pkg.activate("..")              # Project.toml
Pkg.instantiate()               # https://julialang.github.io/Pkg.jl/v1.0/index.html

Activating environment at `~/Documents/Calendar/Work/Noda/Code/Sonic.jl/Project.toml`


In [2]:
import Dates, JuliaDB, Plots, Random

In [3]:
# Random.seed!(Dates.value(Dates.now()))

In [4]:
include("../src/Linckii.jl")
include("../src/LinckiiSecrets.jl")

Main.LinckiiSecrets

Get access token,

In [5]:
access = Linckii.get_access(LinckiiSecrets.keab)

(url = "https://canary.noda.se/~keab", key = "e86d93131e6bc12ae701d036e696cf8b")

GET "https://canary.noda.se/$site_name/api/v1/node"

In [6]:
@time nodes = Linckii.get_nodes(access)

 31.187422 seconds (32.76 M allocations: 1.989 GiB, 2.25% gc time)


Table with 279 rows, 8 columns:
Columns:
#  colname           type
─────────────────────────────────
1  node_public       Bool
2  node_name         String
3  device_id         Int64
4  node_id           Int64
5  node_owner        Bool
6  node_enabled      Bool
7  sensor_ids        Array{Any,1}
8  node_description  String

GET "https://canary.noda.se/$site_name/api/v1/device"

In [7]:
@time devices = Linckii.get_devices(access)

 18.286928 seconds (16.20 M allocations: 1.098 GiB, 1.83% gc time)


Table with 11 rows, 3 columns:
protocol_id  device_name             device_id
──────────────────────────────────────────────
1            "Kelp-IQHeat"           1
2            "Scout"                 2
1            "Kelp-Basic"            4
5            "Weather-Forecast"      6
6            "Grid-Basic"            7
6            "Grid-Controller"       8
7            "Heat-Load-Forecast"    9
2            "CMa1Xw"                10
2            "EcoGuard Sensor"       11
2            "WEPTech"               12
8            "Flexibility-Forecast"  13

GET "https://canary.noda.se/$site_name/api/v1/sensor"

In [8]:
@time sensors = Linckii.get_sensors(access)

 33.835360 seconds (31.45 M allocations: 2.126 GiB, 2.03% gc time)


Table with 175 rows, 5 columns:
Columns:
#  colname             type
────────────────────────────────
1  sensor_unit         FreeUnits
2  sensor_name         Symbol
3  protocol_id         Int64
4  sensor_id           Int64
5  sensor_description  String

Save site,

In [9]:
@time Linckii.savesite(access; nodes = nodes, devices = devices, sensors = sensors)

  2.974772 seconds (7.91 M allocations: 402.344 MiB, 3.94% gc time)


Load site,

In [10]:
@time nodes, devices, sensors = Linckii.loadsite(access, :nodes, :devices, :sensors)

  0.098653 seconds (81.16 k allocations: 4.490 MiB, 18.89% gc time)


(nodes = Table with 279 rows, 8 columns:
Columns:
#  colname           type
─────────────────────────────────
1  node_public       Bool
2  node_name         String
3  device_id         Int64
4  node_id           Int64
5  node_owner        Bool
6  node_enabled      Bool
7  sensor_ids        Array{Any,1}
8  node_description  String, devices = Table with 11 rows, 3 columns:
protocol_id  device_name             device_id
──────────────────────────────────────────────
1            "Kelp-IQHeat"           1
2            "Scout"                 2
1            "Kelp-Basic"            4
5            "Weather-Forecast"      6
6            "Grid-Basic"            7
6            "Grid-Controller"       8
7            "Heat-Load-Forecast"    9
2            "CMa1Xw"                10
2            "EcoGuard Sensor"       11
2            "WEPTech"               12
8            "Flexibility-Forecast"  13, sensors = Table with 175 rows, 5 columns:
Columns:
#  colname             type
───────────────────

Get side 1 substation series,

In [11]:
sensor_names = (
    :meter_effect,
    :meter_primreturntemp,
    :meter_primsupplytemp,
    :meter_volumeflow,
)

(:meter_effect, :meter_primreturntemp, :meter_primsupplytemp, :meter_volumeflow)

In [12]:
signals = JuliaDB.reindex(
    JuliaDB.join(
        Linckii.flatten_nodes(JuliaDB.filter(r -> r.device_id == 4, nodes)),
        JuliaDB.filter(r -> r.sensor_name in sensor_names, sensors);
        lkey=:sensor_id,
        rkey=:sensor_id,
    ),
    (:node_id, :sensor_id),
)

Table with 361 rows, 12 columns:
Columns:
#   colname             type
──────────────────────────────
1   node_id             Int64
2   sensor_id           Int64
3   node_public         Bool
4   node_name           String
5   device_id           Int64
6   node_owner          Bool
7   node_enabled        Bool
8   node_description    String
9   sensor_unit         Any
10  sensor_name         Symbol
11  protocol_id         Int64
12  sensor_description  String

Save data,

In [14]:
dates = Dates.DateTime(2017, 12) : Dates.Month(1) : Dates.DateTime(2019, 12)

2017-12-01T00:00:00:1 month:2019-12-01T00:00:00

In [18]:
rs = JuliaDB.rows(signals)
if false
    rs = [rs[Random.rand(1 : end)]]
end
@time for r in rs
    filepath = "$(Linckii.datapath(access, r.node_id, r.sensor_name)).db"
    if !isfile(filepath)
        try
            Linckii.savedata(access, r.node_id, r.sensor_name, dates...)
            println("$(filepath): Done")
        catch e
            println("$(filepath): $(e)")
        end
    end
end

db/linckii/~keab/data/61/meter_effect.db: ErrorException("column datetime not found.")
 67.265022 seconds (1.03 M allocations: 90.799 MiB)


Load data,

In [16]:
r = JuliaDB.rows(signals)[Random.rand(1 : end)]
(r.node_id, r.sensor_name)

(92, :meter_volumeflow)

In [ ]:
data = Linckii.loaddata(access, r.node_id, r.sensor_name)

Simple plot,

In [ ]:
t = JuliaDB.filter(
    r -> r.datetime >= Dates.DateTime(2018) && r.datetime <= Dates.DateTime(2019),
    data,
)
x = JuliaDB.select(t, :datetime)
y = JuliaDB.select(t, :value)
Plots.plot(x, y, xlabel = :datetime, label = Linckii.datapath(access, r.node_id, r.sensor_name))